In [187]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno


In [188]:
Card= pd.read_csv("Card data.csv")
Bank=pd.read_csv("Open banking data.csv")
Card.info()
Bank.info()
print(Card)
card=Card.drop(columns=['city_name','country_code','intraregion_flag','interregion_flag','merchant' ]) #drop region infos
card.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1379 entries, 0 to 1378
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   cpd_mnth_id            1379 non-null   int64  
 1   cpd_dt                 1379 non-null   object 
 2   cluster_name_adjusted  1379 non-null   object 
 3   mrch_catg_cd           1379 non-null   int64  
 4   mrch_catg_rlup_nm      1378 non-null   object 
 5   merchant               1379 non-null   object 
 6   cp_flag                1379 non-null   int64  
 7   domestic_flag          1379 non-null   int64  
 8   intraregion_flag       1379 non-null   int64  
 9   interregion_flag       1379 non-null   int64  
 10  city_name              1379 non-null   object 
 11  country_code           1379 non-null   int64  
 12  spend                  1379 non-null   float64
dtypes: float64(1), int64(7), object(5)
memory usage: 140.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex:

In [189]:
#Data of people from same country and region

## Card Data Processing 

In [190]:
card.dropna(inplace=True)
card.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1378 entries, 0 to 1378
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   cpd_mnth_id            1378 non-null   int64  
 1   cpd_dt                 1378 non-null   object 
 2   cluster_name_adjusted  1378 non-null   object 
 3   mrch_catg_cd           1378 non-null   int64  
 4   mrch_catg_rlup_nm      1378 non-null   object 
 5   cp_flag                1378 non-null   int64  
 6   domestic_flag          1378 non-null   int64  
 7   spend                  1378 non-null   float64
dtypes: float64(1), int64(4), object(3)
memory usage: 96.9+ KB


In [191]:
card.isnull().sum()

cpd_mnth_id              0
cpd_dt                   0
cluster_name_adjusted    0
mrch_catg_cd             0
mrch_catg_rlup_nm        0
cp_flag                  0
domestic_flag            0
spend                    0
dtype: int64

In [192]:
card.describe()

,cpd_mnth_id,mrch_catg_cd,cp_flag,domestic_flag,spend
count,1378.000000,1378.000000,1378.000000,1378.000000,1378.000000
mean,202306.042816,5926.718433,0.627721,0.960813,28.868186
std,0.814927,917.056110,0.483588,0.194111,36.177928
min,202305.000000,4111.000000,0.000000,0.000000,0.150000
25%,202305.000000,5411.000000,0.000000,1.000000,7.220000
50%,202306.000000,5812.000000,1.000000,1.000000,15.615000
75%,202307.000000,5994.000000,1.000000,1.000000,34.485000
max,202307.000000,9402.000000,1.000000,1.000000,299.980000


In [193]:
card.shape

(1378, 8)

In [194]:
card.nunique()

cpd_mnth_id                3
cpd_dt                    92
cluster_name_adjusted      5
mrch_catg_cd              80
mrch_catg_rlup_nm         80
cp_flag                    2
domestic_flag              2
spend                    932
dtype: int64

From these informations, we can conclure :
1. There are 3 individuals in this survey
2. Time period : 92 days
3. 5 types of transactions
4. 80 types of merchandise and theirs codes corresponding


In [195]:
card["cpd_dt"] = pd.to_datetime(card["cpd_dt"])
card.rename(columns={"cpd_dt": "date",'cpd_mnth_id':'ID','cluster_name_adjusted': 'Type_of_transaction','mrch_catg_rlup_nm':'Merchandise_catg','mrch_catg_cd':'Merchandise_code'}, inplace=True)
card.head()

,ID,date,Type_of_transaction,Merchandise_code,Merchandise_catg,cp_flag,domestic_flag,spend
0,202306,2023-06-08,Family,7523,"PARKING LOTS,METERS,GARAGES",0,1,6.25
1,202306,2023-06-13,Affluent Segment,5499,MISC FOOD STORES - DEFAULT,1,1,1.39
2,202307,2023-07-09,Affluent Segment,5499,MISC FOOD STORES - DEFAULT,1,1,1.41
3,202306,2023-06-13,Affluent Segment,5499,MISC FOOD STORES - DEFAULT,1,1,1.39
4,202307,2023-07-09,Affluent Segment,5499,MISC FOOD STORES - DEFAULT,1,1,1.41


In [196]:
#card['Merchandise']= card.Merchandise_code.astype(str).str.cat(card.Merchandise_catg.astype(str), sep='-')
#card.head()
card=card.drop(columns=['cp_flag', 'domestic_flag'])

In [197]:
card.head()

,ID,date,Type_of_transaction,Merchandise_code,Merchandise_catg,spend
0,202306,2023-06-08,Family,7523,"PARKING LOTS,METERS,GARAGES",6.25
1,202306,2023-06-13,Affluent Segment,5499,MISC FOOD STORES - DEFAULT,1.39
2,202307,2023-07-09,Affluent Segment,5499,MISC FOOD STORES - DEFAULT,1.41
3,202306,2023-06-13,Affluent Segment,5499,MISC FOOD STORES - DEFAULT,1.39
4,202307,2023-07-09,Affluent Segment,5499,MISC FOOD STORES - DEFAULT,1.41


So assume these 3 individuals represent for a group of population in a specific region, we can split this data into 3 different study case :
1. Based on the ID
2. Based on type of transactions
3. Based on the merchadise category 

In [264]:
# return data to csv file
card.to_csv('../Visa Datathon/Clean data/Card_data_clean.csv', index=False)

## Bank account data processing 

In [199]:
Bank.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1378 entries, 0 to 1377
Data columns (total 8 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Name                       1378 non-null   object 
 1   Value.accountId            1378 non-null   object 
 2   cluster_name_adjusted      1378 non-null   object 
 3   Value.amount.currencyCode  1378 non-null   object 
 4   mrch_catg_rlup_nm2         1378 non-null   object 
 5   Value.dates.booked         1378 non-null   object 
 6   amount                     1378 non-null   float64
 7   Category                   1372 non-null   object 
dtypes: float64(1), object(7)
memory usage: 86.3+ KB


In [200]:
Bank.head()
#Bank.tail()

,Name,Value.accountId,cluster_name_adjusted,Value.amount.currencyCode,mrch_catg_rlup_nm2,Value.dates.booked,amount,Category
0,transactions,5a73582adf954cf6b3db6cc97bedccd9,Affluent Segment,GBP,AMUSEMENT PARKS/CIRCUS,5/12/2023,3.73,entertainment
1,transactions,5a73582adf954cf6b3db6cc97bedccd9,Affluent Segment,GBP,AMUSEMENT PARKS/CIRCUS,5/12/2023,3.73,entertainment
2,transactions,5a73582adf954cf6b3db6cc97bedccd9,Affluent Segment,GBP,AMUSEMENT PARKS/CIRCUS,6/2/2023,4.39,entertainment
3,transactions,5a73582adf954cf6b3db6cc97bedccd9,Affluent Segment,GBP,AMUSEMENT PARKS/CIRCUS,6/2/2023,7.40,entertainment
4,transactions,6r88582adf954cf6b3db6cc97bedccd9,Urban Spenders,GBP,ART DEALERS & GALLERIES,7/14/2023,6.57,entertainment


In [201]:
Bank.nunique()

Name                           1
Value.accountId                5
cluster_name_adjusted          5
Value.amount.currencyCode      1
mrch_catg_rlup_nm2            80
Value.dates.booked            92
amount                       932
Category                       7
dtype: int64

In [202]:
Bank['Value.dates.booked']= pd.to_datetime(Bank['Value.dates.booked'])

In [203]:
bank=Bank.drop(columns=['Name','Value.amount.currencyCode'])
#Only one type of currency and same transaction, so we can drop these infomations

In [204]:
bank.dropna(inplace=True)
bank.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1372 entries, 0 to 1377
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Value.accountId        1372 non-null   object        
 1   cluster_name_adjusted  1372 non-null   object        
 2   mrch_catg_rlup_nm2     1372 non-null   object        
 3   Value.dates.booked     1372 non-null   datetime64[ns]
 4   amount                 1372 non-null   float64       
 5   Category               1372 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 75.0+ KB


In [205]:
bank.nunique()

Value.accountId            5
cluster_name_adjusted      5
mrch_catg_rlup_nm2        78
Value.dates.booked        92
amount                   929
Category                   7
dtype: int64

In [206]:
bank.rename(columns={"Value.accountId":"accountID","cluster_name_adjusted":"spending_degree","mrch_catg_rlup_nm2":"merchandise_catg","Val.dates.booked":"date"},inplace=True)
bank.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1372 entries, 0 to 1377
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   accountID           1372 non-null   object        
 1   spending_degree     1372 non-null   object        
 2   merchandise_catg    1372 non-null   object        
 3   Value.dates.booked  1372 non-null   datetime64[ns]
 4   amount              1372 non-null   float64       
 5   Category            1372 non-null   object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 75.0+ KB


In [233]:
bank['accountID'].unique()

array(['5a73582adf954cf6b3db6cc97bedccd9',
       '6r88582adf954cf6b3db6cc97bedccd9',
       '5a73582adf954cf6b3db6cc97tedggd9',
       '94177e7a3daa4ef18746b355980ebd5f',
       '5a73582adf954cf6b3db6cc97yedood7'], dtype=object)

In [242]:
input= bank['accountID']
output=[]
for i in input:
    output.append('account_{}'.format(i[0:3]+i[-3:-1]))

output

['account_5a7cd',
 'account_5a7cd',
 'account_5a7cd',
 'account_5a7cd',
 'account_6r8cd',
 'account_6r8cd',
 'account_5a7gd',
 'account_5a7gd',
 'account_6r8cd',
 'account_5a7cd',
 'account_5a7cd',
 'account_5a7cd',
 'account_941d5',
 'account_5a7gd',
 'account_5a7gd',
 'account_5a7od',
 'account_5a7od',
 'account_5a7od',
 'account_5a7od',
 'account_5a7od',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_5a7gd',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_941d5',
 'account_5a7od',
 'account_

In [255]:
bank['accountID']=output
bank.head()

,accountID,spending_degree,merchandise_catg,Value.dates.booked,amount,Category
0,account_5a7cd,Affluent Segment,AMUSEMENT PARKS/CIRCUS,2023-05-12,3.73,entertainment
1,account_5a7cd,Affluent Segment,AMUSEMENT PARKS/CIRCUS,2023-05-12,3.73,entertainment
2,account_5a7cd,Affluent Segment,AMUSEMENT PARKS/CIRCUS,2023-06-02,4.39,entertainment
3,account_5a7cd,Affluent Segment,AMUSEMENT PARKS/CIRCUS,2023-06-02,7.40,entertainment
4,account_6r8cd,Urban Spenders,ART DEALERS & GALLERIES,2023-07-14,6.57,entertainment


In [257]:
bank['merchandise_catg'].unique()

array(['AMUSEMENT PARKS/CIRCUS', 'ART DEALERS & GALLERIES',
       'ARTIST/CRAFT SHOPS', 'AUTOMATED FUEL DISPENSERS',
       'AUTOMOTIVE TIRE STORES', 'BAKERIES', 'BANDS/ORCHESTRAS/ENTERTAIN',
       'BARS/TAVERNS/LOUNGES/DISCOS', 'BEAUTY/BARBER SHOPS',
       'BOOK STORES', 'BUS LINES', 'BUYING/SHOPPING SERVICES',
       'CABLE, SAT, PAY TV/RADIO SVCS', 'CATERERS',
       'CHARITABLE/SOC SERVICE ORGS', 'CHILDREN/INFANTS WEAR STORES',
       'CIGAR STORES/STANDS', 'COMPUTER SOFTWARE STORES',
       'CONTINUITY/SUBSCRIPTION MERCHT', 'COURIER SERVICES',
       'DENTISTS/ORTHODONTISTS', 'DEPARTMENT STORES',
       'DIGITAL GOODS BOOKSMOVIEMUSIC', 'DIGITAL GOODS GAMES',
       'DISCOUNT STORES', 'DRUG STORES & PHARMACIES',
       'ELECTRONICS STORES', 'FAMILY CLOTHING STORES',
       'FAST FOOD RESTAURANTS', 'FLORISTS', 'FREEZER/MEAT LOCKERS',
       'GIFT, CARD, NOVELTY STORES', "GOV'T SERV - DEFAULT",
       'GROCERY STORES/SUPERMARKETS', 'HARDWARE STORES',
       'HOME SUPPLY WAREHOUSE 

In [258]:
bank['Category'].unique()

array(['entertainment', 'transport', 'food', 'wellness', 'shopping',
       'home', 'misc'], dtype=object)

In [263]:
bank.to_csv('../Visa Datathon/Clean data/Bank_data_clean.csv', index=False)